# City similarity [ Bangkok - Kyoto ]

## Import zone

In [1]:
import gensim
import os
import collections
import smart_open
import random
import pickle
import pandas as pd
import copy
import re

C:\Users\sociocom\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Parameter zone

In [2]:
path_play = r'C:\Users\sociocom\Desktop\Crawler\backup\playing'

In [280]:
grid_size = 10
diff = 0.1/grid_size

In [281]:
bkk_lat = 13.685780
bkk_long = 100.484605
diff_wid = 0.1/grid_size

In [53]:
kyt_lat = 34.955205 #35.005205
kyt_long = 135.675300 #135.7353
diff_wid = (135.795300 - 135.675300)/grid_size
#135.675300,34.955205,135.795300,35.055205

In [6]:
#merge to 10 color
color_idx = {}
idx_color = {}

In [81]:
boundary = 20

In [33]:
df_kyt.head()

,screen_name,id_user_str,id_str,text,created_at,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,ex_lat,ex_long,lot_ne,lot_sw,place
0,anti_satsuki,903659715921567744,1002203935950692352,"I'm at JR Kyoto in Kyoto, Kyoto https://t.co/7...",Thu May 31 15:03:33 +0000 2018,None,None,None,ja,34.985353,135.758295,"135.778292, 35.000701","135.734405, 34.977134","Kamigyo-ku, Kyoto-shi"
1,syohe8027,2282624300,1002204274707886082,I'm at Starbucks Coffee Kyoto Station Store - ...,Thu May 31 15:04:53 +0000 2018,None,None,None,ja,34.995875,135.738674,"136.987745, 35.779334","134.252598, 33.433256",Kinki area
2,Wish__AZ,2741161340,1002204735355674624,I'm at Ramen Yokozuna Izukaji shop in Kyoto Ci...,Thu May 31 15:06:43 +0000 2018,None,None,None,ja,35.041635,135.782933,"135.860702, 35.32117","135.725501, 35.00634",Sakyo
3,mgn_kyoto,487389327,1002206619072004096,【24: 00 time point 17.5 ℃ 92%】 21.3 to 16.3 ° ...,Thu May 31 15:14:12 +0000 2018,None,None,None,ja,35.013333,135.731667,"135.779805, 35.018685","135.730513, 34.99342",Kyoto City Nakagyo-ku
4,amds_kyotoKyoto,482601329,1002208198625869825,Kyoto (Kyoto) 24:00 Temperature 17.5 ℃ West an...,Thu May 31 15:20:29 +0000 2018,None,None,None,ja,35.013333,135.731667,"135.779805, 35.018685","135.730513, 34.99342",Kyoto City Nakagyo-ku


## Function zone

In [8]:
def save_obj(file, obj):
    with open(file, 'wb') as f:
        pickle.dump(obj, f)

def load_obj(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [9]:
def clean_place(df, name):
    return df[df['place']!=name]

In [56]:
def separate_loc(df, flat, flong):
    df['ex_lat'] = df['ex_lat'].apply(lambda x: float(x))
    df['ex_long'] = df['ex_long'].apply(lambda x: float(x))
    data = []
    lat = flat
    long = flong
    for i in range(grid_size):
        for j in range(grid_size):
            data.append(df[(df['ex_lat'] >= lat) & (df['ex_lat'] < lat+diff) & (df['ex_long'] >= long) & (df['ex_long'] < long + diff_wid)])
            idx = []
            idx.append((long, lat))
            idx.append((long+diff_wid, lat))
            #print(lat," ",long)
            lat += diff
            idx.append((long, lat))
            idx.append((long+diff_wid, lat))
            u = i*10 + j
            idx.append(str(u))
            idx_color[str(u)] = idx
        long += diff_wid
        lat = flat
    return data

In [11]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="utf-8") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [12]:
def in_tuple(tp, x):
    return [z for y,z in tp if y == x]

In [300]:
def merge_area(threshold, idx_color,city_list):
    for i in range(len(idx_color)):
        w = i-1 if (i-1)%10 != 9 else -1
        e = i+1 if ((i+1)%10 != 0) and (i+1 < len(idx_color)) else -1
        n = i-10 
        s = i+10 if i+10 < len(idx_color) else -1
        if len(city_list[i])<boundary:
            idx_color[str(i)][4] = "-1"
        else:
            if w>0 and len(city_list[w])>=boundary:
                similarw = (in_tuple(all_sims[i], w)[0] + in_tuple(all_sims[w], i)[0])/2
                if similarw > threshold:
                    idx_color[str(w)][4] = idx_color[str(i)][4]
            if e>0 and len(city_list[e])>=boundary:
                similare = (in_tuple(all_sims[i], e)[0] + in_tuple(all_sims[e], i)[0])/2
                if similare > threshold:
                    idx_color[str(e)][4] = idx_color[str(i)][4]
            if n>0 and len(city_list[n])>=boundary:
                similarn = (in_tuple(all_sims[i], n)[0] + in_tuple(all_sims[n], i)[0])/2
                if similarn > threshold:
                    idx_color[str(n)][4] = idx_color[str(i)][4]
            if s>0 and len(city_list[s])>=boundary:
                similars = (in_tuple(all_sims[i], s)[0] + in_tuple(all_sims[s], i)[0])/2
                if similars > threshold:
                    idx_color[str(s)][4] = idx_color[str(i)][4]
    return idx_color

In [14]:
def count_areas(idx_color):
    areas = set()
    for i in range(10):
        for j in range(10):
            areas.add(idx_color[str((i*10)+j)][4])
    return len(areas)

In [15]:
def print_areas(idx_color):
    areas = set()
    for i in range(10):
        s = ""
        for j in range(10):
            idx = str((i*10)+j)
            if int(idx_color[idx][4]) < 10 and int(idx_color[idx][4]) >= 0:
                s += " "
            s += str(idx_color[idx][4]) +" "
            areas.add(idx_color[idx][4])
        print(s)
    print("\n #areas :" , len(areas))
    return areas

In [16]:
def print_volumn(list_place):
    for i in range(10):
        s = ""
        for j in range(10):
            idx = (i*10)+j
            if len(list_place[idx]) < 99 and len(list_place[idx]) >= 0:
                s += " "
                if len(list_place[idx]) <10:
                    s+= " "
            s += str(len(list_place[idx])) +" "
        print(s)

In [17]:
#no
def clean_reply(df):
    for i in range(len(df)):
        if (df["ex_lat"][i] == -1) | (df["tweet_reply_username"][i] == -1) | (df["tweet_reply_username"][i] == None):
            continue
        idx_reply = df["text"][i].find(df["tweet_reply_username"][i])-1
        leng = len(df["tweet_reply_username"][i])+1
        df.at[i, "text"] = df["text"][i][:idx_reply] + " <MENTION> " + df["text"][i][idx_reply+leng:]

In [18]:
#no
def clean_url(city):
    city_str = []
    for y in city:
        doc = ""
        for x in y['text']:
            print(x)
            if "https://" in x.lower():
                idx = (x.lower().find("https://"))-1
                x = x[:idx] + " <URL>"
            if "I'm at " in x and "<URL>" in x:
                idx_i = (x.find("I'm at "))
                idx_url = (x.find("<URL>"))+5
                x = x[:idx_i]+x[idx_url:]+"<CHECK_IN>"
            doc += x + ". "
        doc += " \n"
        city_str.append(doc)
        return city_str

## Data zone

## Kyoto 

In [118]:
df_kyt = load_obj(path_play + r"\kyoto_translated1")

In [119]:
df_kyt["place"].value_counts()

Kyoto City Nakagyo-ku                                                  13087
Kamigyo-ku, Kyoto-shi                                                   8894
Kamigyo-ku, Kyoto City                                                  3905
Sakyo                                                                   3046
Kami-ku                                                                 2086
Kyoto City North District                                               1415
Kyoto-shi Nakagyo                                                       1314
Kinki area                                                              1296
Kyoto City South District                                               1253
Kyoto City Area                                                         1174
Fushimi-ku, Kyoto-shi                                                    715
Kyoto City, Dongshan District                                            652
Kyoto City Shimogyo Ward                                                 581

In [120]:
df_kyt.head()

,screen_name,id_user_str,id_str,text,created_at,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,ex_lat,ex_long,lot_ne,lot_sw,place
0,anti_satsuki,903659715921567744,1002203935950692352,"I'm at JR Kyoto in Kyoto, Kyoto https://t.co/7...",Thu May 31 15:03:33 +0000 2018,None,None,None,ja,34.985353,135.758295,"135.778292, 35.000701","135.734405, 34.977134","Kamigyo-ku, Kyoto-shi"
1,syohe8027,2282624300,1002204274707886082,I'm at Starbucks Coffee Kyoto Station Store - ...,Thu May 31 15:04:53 +0000 2018,None,None,None,ja,34.995875,135.738674,"136.987745, 35.779334","134.252598, 33.433256",Kinki area
2,Wish__AZ,2741161340,1002204735355674624,I'm at Ramen Yokozuna Izukaji shop in Kyoto Ci...,Thu May 31 15:06:43 +0000 2018,None,None,None,ja,35.041635,135.782933,"135.860702, 35.32117","135.725501, 35.00634",Sakyo
3,mgn_kyoto,487389327,1002206619072004096,【24: 00 time point 17.5 ℃ 92%】 21.3 to 16.3 ° ...,Thu May 31 15:14:12 +0000 2018,None,None,None,ja,35.013333,135.731667,"135.779805, 35.018685","135.730513, 34.99342",Kyoto City Nakagyo-ku
4,amds_kyotoKyoto,482601329,1002208198625869825,Kyoto (Kyoto) 24:00 Temperature 17.5 ℃ West an...,Thu May 31 15:20:29 +0000 2018,None,None,None,ja,35.013333,135.731667,"135.779805, 35.018685","135.730513, 34.99342",Kyoto City Nakagyo-ku


In [121]:
df_kyt = clean_place(df_kyt, "Japan")
df_kyt = clean_place(df_kyt, "Kyoto")

In [122]:
df_kyt.reset_index(drop="index", inplace=True)

In [123]:
#df_kyt.rename(columns={"in_reply_to_screen_name":"tweet_reply_username"}, inplace=True)

In [124]:
#clean_reply(df_kyt)

In [125]:
kyt = separate_loc(df_kyt, kyt_lat, kyt_long)

In [126]:
#kyt_str = clean_url(kyt)
kyt_str = []
for y in kyt:
    doc = ""
    for x in y['text']:
        if "https://" in x.lower():
            idx = (x.lower().find("https://"))-1
            x = x[:idx] + " <URL>"
        if "I'm at " in x and "<URL>" in x:
            idx_i = (x.find("I'm at "))
            idx_url = (x.find("<URL>"))+5
            x = x[:idx_i]+x[idx_url:]+"<CHECK_IN>"
        doc += x + ". "
    doc += " \n"
    kyt_str.append(doc)

In [127]:
f = open(path_play + "/kyt_demo_train1","w",encoding="utf-8")
#file = open(path + "bkk_demo_test","w",encoding="utf-8")
for x in kyt_str:
    f.write(x)
    print(x)
f.close()

I 'm at Lucksine in Kyoto City, Kyoto Prefecture <URL>. Was it a little early? (@ FamilyMart Daido Tozai Machi in Kyoto City, Kyoto Prefecture) <URL>. <CHECK_IN>. I 'm at Lucksine in Kyoto City, Kyoto Prefecture <URL>. <CHECK_IN>. I 'm at Lucksine in Kyoto City, Kyoto Prefecture <URL>. I 'm at Lucksine in Kyoto City, Kyoto Prefecture <URL>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. I 'm at Lucksine in Kyoto City, Kyoto Prefecture <URL>. Prescription and a bit of shopping (@ Ducks Rock West Central store in Kyoto City, Kyoto Prefecture) <URL>. Bamboo Road (Difference) (@ Lukanshi Bamboo Park in Kyoto City, Kyoto Prefecture) <URL>. <CHECK_IN>.  

<CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. Katsura is Miruki & amp; Koho's W's visit ~ w  # Saito Aya # Aoi Kaoho # Sunny Day Girls ... <URL>. Were kako 's blue blue only Aoi? ! w  # Aoi Kaho # Sunny Day Girls ... <URL>. Milky's orange has nothing due to Mirukie w  # Saito Aya # Sunny Day Girls ... <URL>. Here too well Watchawacha friends photo (

In [128]:
len(kyt_str)

100

In [67]:
kyt_str[0]

"I 'm at Lucksine in Kyoto City, Kyoto Prefecture <URL>. Was it a little early? (@ FamilyMart Daido Tozai Machi in Kyoto City, Kyoto Prefecture) <URL>. <CHECK_IN>. I 'm at Lucksine in Kyoto City, Kyoto Prefecture <URL>. <CHECK_IN>. I 'm at Lucksine in Kyoto City, Kyoto Prefecture <URL>. I 'm at Lucksine in Kyoto City, Kyoto Prefecture <URL>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. I 'm at Lucksine in Kyoto City, Kyoto Prefecture <URL>. Prescription and a bit of shopping (@ Ducks Rock West Central store in Kyoto City, Kyoto Prefecture) <URL>. Bamboo Road (Difference) (@ Lukanshi Bamboo Park in Kyoto City, Kyoto Prefecture) <URL>. <CHECK_IN>.  \n"

### clean data

In [233]:
#df_bkk1 = load_obj(path_play + r"\translated_bkk_tweet1")
#df_bkk2 = load_obj(path_play + r"\translated_bkk_tweet2")

In [234]:
list_df = []
for filename in os.listdir(path_play+"\\bkk"):
    with open(path_play + "\\bkk\\" + filename, "rb") as f:
        in_path = path_play + "\\bkk\\" + filename
        print(in_path)
        df = load_obj(in_path)
        df.reset_index(inplace=True, drop="index")
        list_df.append(df)

C:\Users\sociocom\Desktop\Crawler\backup\playing\bkk\translated_bkk_tweet1
C:\Users\sociocom\Desktop\Crawler\backup\playing\bkk\translated_bkk_tweet2
C:\Users\sociocom\Desktop\Crawler\backup\playing\bkk\translated_bkk_tweet3
C:\Users\sociocom\Desktop\Crawler\backup\playing\bkk\translated_bkk_tweet4


In [235]:
df_bkk = pd.concat(list_df)

In [236]:
df_bkk.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,132398,132398,132398,132398,132398,132398,132398,132398,132398,132398,132398,132398,132398,132398
unique,17199,17288,132398,114134,124002,5640,5620,154,108,106,31552,15468,15489,2
top,882432466526453760,404notdown,1005452702200025088,I do not know what to do.,2018-06-03 09:18:20,-1,-1,Thailand,"100.528375, 13.72951","100.541027, 13.749708",-1,-1,-1,th
freq,669,669,1,1650,5,117490,117490,9420,9837,9837,99031,97451,97451,109077


In [237]:
df_bkk = clean_place(df_bkk, "Myanmar")
df_bkk = clean_place(df_bkk, "Thailand")
df_bkk = clean_place(df_bkk, "Bangkok")

In [238]:
df_bkk.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,113992,113992,113992,113992,113992,113992,113992,113992,113992,113992,113992,113992,113992,113992
unique,14369,14431,113992,97663,107745,5307,5287,151,105,103,27705,13327,13345,2
top,882432466526453760,404notdown,1005452702200025088,I do not know what to do.,2018-06-08 06:27:52,-1,-1,Pathum Wan,"100.528375, 13.72951","100.541027, 13.749708",-1,-1,-1,th
freq,669,669,1,1506,5,100012,100012,6227,9837,9837,85289,84043,84043,96966


In [239]:
len(df_bkk[(df_bkk["ex_lat"]!= -1) & (df_bkk["tweet_reply_id"]!= "-1")])

28703

In [240]:
df_bkk.reset_index(inplace=True)

In [241]:
df_bkk["text"][1]

'Walk through (@Sam Market Market (Wat Phra Kaew Market) in Bang Rak, Bangkok) https://t.co/7TsvbPXdGE'

In [242]:
for i in range(len(df_bkk)):
    if (df_bkk["ex_lat"][i] == -1) | (df_bkk["tweet_reply_username"][i] == -1):
        continue
    idx_reply = df_bkk["text"][i].find(df_bkk["tweet_reply_username"][i])-1
    leng = len(df_bkk["tweet_reply_username"][i])+1
    df_bkk.at[i, "text"] = df_bkk["text"][i][:idx_reply] + " <MENTION> " + df_bkk["text"][i][idx_reply+leng:]

In [243]:
df_bkk.head()

,index,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
0,0,137552452,ExpatDanBKK,1001726470962769920,"<MENTION> Hmmm, I suspect Bill Heinecke woul...",2018-05-30 07:26:16,13.742844,100.5682602,Vadhana,"100.55007, 13.732979","100.570733, 13.74842",1001725702192824321,710742220681293824,David_Aamez,en
1,1,372674276,NeungBellerin,1001726500725456897,Walk through (@Sam Market Market (Wat Phra Kae...,2018-05-30 07:26:23,13.724922,100.52291,Suriya Wong,"100.516253, 13.72236","100.536283, 13.730574",-1,-1,-1,th
2,2,1030302462,opalopongpang,1001726547160690689,I'm at Penguin Eat Shabu in Vadhana https://t....,2018-05-30 07:26:34,13.7388373,100.56067598,Vadhana,"100.55007, 13.732979","100.570733, 13.74842",-1,-1,-1,en
3,3,953277866661199872,Poppysuphannee,1001726564550168576,And it's not going to be a school ride. I do n...,2018-05-30 07:26:38,-1,-1,Frown,"100.469033, 13.773245","100.49389, 13.789877",-1,-1,-1,th
4,4,399872095,StoneSoupBKK,1001726646599041024,<MENTION> F@$king morons.,2018-05-30 07:26:58,-1,-1,Phra Khanong Nuea,"100.584345, 13.704628","100.606727, 13.729193",1001701922074701824,774174489458331649,TheMossadIL,en


In [244]:
bkk = separate_loc(df_bkk, bkk_lat, bkk_long)

In [245]:
idx_color["3"]

[(100.484605, 13.715779999999999),
 (100.494605, 13.715779999999999),
 (100.484605, 13.725779999999999),
 (100.494605, 13.725779999999999),
 '3']

In [246]:
count = 0
valid = 0
for i in range(len(bkk)):
    count += len(bkk[i])
    if len(bkk[i])>20:
        valid += 1
    print("#tweets in area",i,":" ,len(bkk[i]))
print(count)
print(valid)

#tweets in area 0 : 3
#tweets in area 1 : 15
#tweets in area 2 : 12
#tweets in area 3 : 45
#tweets in area 4 : 11
#tweets in area 5 : 63
#tweets in area 6 : 1122
#tweets in area 7 : 84
#tweets in area 8 : 38
#tweets in area 9 : 23
#tweets in area 10 : 6
#tweets in area 11 : 40
#tweets in area 12 : 8
#tweets in area 13 : 60
#tweets in area 14 : 27
#tweets in area 15 : 40
#tweets in area 16 : 170
#tweets in area 17 : 341
#tweets in area 18 : 60
#tweets in area 19 : 45
#tweets in area 20 : 3
#tweets in area 21 : 20
#tweets in area 22 : 23
#tweets in area 23 : 76
#tweets in area 24 : 69
#tweets in area 25 : 239
#tweets in area 26 : 46
#tweets in area 27 : 31
#tweets in area 28 : 42
#tweets in area 29 : 55
#tweets in area 30 : 3
#tweets in area 31 : 17
#tweets in area 32 : 16
#tweets in area 33 : 120
#tweets in area 34 : 71
#tweets in area 35 : 102
#tweets in area 36 : 40
#tweets in area 37 : 11
#tweets in area 38 : 13
#tweets in area 39 : 31
#tweets in area 40 : 10
#tweets in area 41 : 10


In [247]:
idx_color["56"] #Siam

[(100.53460500000003, 13.745779999999998),
 (100.54460500000003, 13.745779999999998),
 (100.53460500000003, 13.755779999999998),
 (100.54460500000003, 13.755779999999998),
 '56']

### prepare data

In [248]:
bkk_str = []
for y in bkk:
    doc = ""
    for x in y['text']:
        if "https://" in x.lower():
            idx = (x.lower().find("https://"))-1
            x = x[:idx] + " <URL>"
        if "I'm at " in x and "<URL>" in x:
            idx_i = (x.find("I'm at "))
            idx_url = (x.find("<URL>"))+5
            x = x[:idx_i]+x[idx_url:]+"<CHECK_IN>"
        doc += x + ". "
    doc += " \n"
    bkk_str.append(doc)

In [249]:
#for y in bkk_str:
#    pattern2.sub("<CHECK_IN>", y)
#    print(y)

In [250]:
print(bkk_str[25])

<CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. Near Wat Lung Na Yi (Yaowarat in Samphanthawong, Bangkok) <URL>. I'm new here (@Yellow Shopping Center in Bangkok, Thailand) <URL>. <CHECK_IN>. <CHECK_IN>. I'm sorry for the dela <MENTION> 'm sorry for the delay.. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. I'm not sure if this is the right place for yo <MENTION> 'm not sure if this is the right place for you.. But I eat rice (@ Duck Noodle in Samphanthawong, Bangkok) <URL>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. ((((((Https https https https https https https https https https https https https https https https https https https https https https htt <MENTION> (((((Https https https https https https https https https https https https https https https https https https https https https https https. <CHECK_

In [251]:
f = open(path_play + "/bkk_demo_train3","w",encoding="utf-8")
#file = open(path + "bkk_demo_test","w",encoding="utf-8")
for x in bkk_str:
    f.write(x)
    print(x)
f.close()

Hello, special room @ ... <URL>. I do not think so. <URL>. เอิ้วววววววววว @ The View Bar &amp; Restaurant <URL>.  

<CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. Just posted a photo @ ... <URL>. Today Today #Bangkok #Tailandia #Tromboranga ... <URL>. Just posted a photo… <URL>. In Thailand thinking about life. Great ppl,… <URL>.  Beef Tenderloin Roasted Whole And Served… <URL>. Just posted a photo @ AVANI Riverside Bangkok… <URL>. What do you eat? <URL>.  

<CHECK_IN>. <CHECK_IN>. The first dental implant (@ dentist dentist) <URL>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. Today Friday ... <URL>. The album is coming for a while ... <URL>. I'm sorry @ Buta Shubu Tha Bright <URL>. Whirlpool (@ Somdejprapinklao Hospital in Thon Buri, Bangkok) <URL>. # 13062018 @ My Home Flats ... <URL>. We do not sleep (OO). <URL>.  

This #job might be a great fit for you: HR Associate Manager - Thailand - <URL>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. 

## Doc2Vec

In [301]:
train_corpus = list(read_corpus(path_play + "/bkk_demo_train3"))

In [302]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=80, min_count=2, epochs=100, dm=0)
#model = gensim.models.doc2vec.Doc2Vec(vector_size=80, min_count=2, epochs=100)

In [303]:
model.build_vocab(train_corpus)

In [304]:
#model.wv.vocab["สยาม"].count

In [305]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

Wall time: 2.82 s


In [306]:
ranks = []
second_ranks = []
all_sims = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    all_sims.append(sims)
    
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [307]:
print(ranks,"\n")
print(ranks.count(0))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 69, 0, 0, 0, 0, 0, 0, 0, 0] 

97


In [308]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (99): «virginstar listen to the song and tell feeling url check_in dpcynl cattshirt cattshirt catradio chayanit patchayanit url just began running workout using endomondo peptalk me now url virginhitz and listen to the song feeling url virginhitz feel free to listen and tell feeling url check_in just began running workout using endomondo peptalk me now url just began running workout using endomondo peptalk me now url just finished running km in with endomondo endorphins url hope to get this kind of view again soon url looking forward to see this beauty in person url check_in do not know what to do url just began running workout using endomondo peptalk me now url here am here am coffee amp bingsu url virginhitz love music and say feeling url jal mtfuji heading to itami airport url check_in over the cloud there is beautiful sunrise url time url check_in just began running workout using endomondo peptalk me now url over the cloud there is beautiful sunrise url fujisan in japanese

## Merge area

In [309]:
idx_color2 = copy.deepcopy(idx_color)

In [310]:
print_areas(idx_color)

 0  1  2  3  4  5  6  7  8  9 
10 11 12 13 14 15 16 17 18 19 
20 21 22 23 24 25 26 27 28 29 
30 31 32 33 34 35 36 37 38 39 
40 41 42 43 44 45 46 47 48 49 
50 51 52 53 54 55 56 57 58 59 
60 61 62 63 64 65 66 67 68 69 
70 71 72 73 74 75 76 77 78 79 
80 81 82 83 84 85 86 87 88 89 
90 91 92 93 94 95 96 97 98 99 

 #areas : 100


{'0',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '6',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '8',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '9',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99'}

In [311]:
threshold = 0.7
areas = count_areas(idx_color2)
while areas > 11:
    idx_color2 = merge_area(threshold, idx_color2, bkk)
    threshold -= 0.001
    areas = count_areas(idx_color2)
print(threshold)

0.23999999999999955


In [312]:
print_areas(idx_color2)

-1 -1 -1  3 -1  5  5  7  7  7 
-1 11 -1  3  3  3 16 16 16 16 
-1 11 11  3  3  3  3  3 16 16 
-1 -1 -1  3  3  3  3 -1 -1 16 
-1 -1 42 42 42 42 42 42 42 -1 
-1 51 42 42 42 42 42 42 42 42 
-1 -1 -1 63 63 63 -1 -1 42 42 
-1 -1 -1 63 63 63 63 77 -1 42 
-1 -1 -1 63 63 63 63 63 63 63 
-1 -1 92 92 63 63 63 -1 63 63 

 #areas : 11


{'-1', '11', '16', '3', '42', '5', '51', '63', '7', '77', '92'}

In [294]:
print_volumn(bkk)

  3  15  12  45  11  63 1122  84  38  23 
  6  40   8  60  27  40 170 341  60  45 
  3  20  23  76  69 239  46  31  42  55 
  3  17  16 120  71 102  40  11  13  31 
 10  10  24 238 272 1262 874  75 105  15 
 11  77  44  70 296 430 1453 363  51 104 
 11   3   4  27  40 280  15   8  38  61 
  0   1   7 108 102 714 131  24   8  71 
  5   2   2  61 419 156 144 236 491  30 
  1   0  31 119 159  34  41   4  22  27 


In [140]:
print_areas(idx_color2)

-1  1  1  1  1  1  1 -1 -1 -1 
-1  1  1  1  1  1  1 -1 18 -1 
20 -1 20 20 20 20 20 -1 -1 -1 
20 20 20 20 20 20 20 20 -1 -1 
-1 50 50 20 53 20 20 20 20 20 
50 50 50 53 53 20 20 20 20 20 
50 50 50 63 64 20 20 20 20 20 
50 50 50 63 74 75 76 20 50 50 
50 50 50 50 50 50 50 50 50 50 
-1 50 -1 50 50 50 50 50 50 50 

 #areas : 11


{'-1', '1', '18', '20', '50', '53', '63', '64', '74', '75', '76'}

In [141]:
print_volumn(kyt)

 14  44  28  24  30 276 191  15   1   8 
 10 153  44  34  46  32  54   9 158   0 
246  15 236  44  28  50  76  17   1   0 
 42  29  42 398 244 125  68  87   2   1 
 10  78 215  84 618 2387 225 280 214  20 
137 124 109 462 1017 772 644 182 109 123 
337 160 3282 3827 2835 458 1008 221 182 155 
316 269 587 1066 2785 3692 302 471 123 142 
 37 525  51 156 1337 1145 329 2374 217  60 
  2  37  11 208  31 345  94 304 437 330 


In [142]:
print(str(list(idx_color["0"][0])))

[135.6753, 34.955205]


In [295]:
map_color = {}
#color = ['#17263c','#c9b2a6','#dcd2be','#ae9e90','#dfd2ae','#dfd2ae', '#93817c','#f8c967','#8f7d77','#dfd2ae','#b9d3c2']
color = ['white','orange','gray','black','brown','pink', 'purple','yellow','red','green','blue']

In [296]:
for i in range(11):
    map_color[str(i)]=color[i]

In [313]:
map_color

{'0': 'white',
 '1': 'orange',
 '10': 'blue',
 '2': 'gray',
 '3': 'black',
 '4': 'brown',
 '5': 'pink',
 '6': 'purple',
 '7': 'yellow',
 '8': 'red',
 '9': 'green'}

In [314]:
idx_color2["0"]

[(100.484605, 13.68578),
 (100.494605, 13.68578),
 (100.484605, 13.69578),
 (100.494605, 13.69578),
 '-1']

In [315]:
f_loc = open("latlong_bkk_dbow2.txt","w", encoding="utf-8")
color_area = list(print_areas(idx_color2))
color_area.sort()
f_loc.write("""{
  "type": "FeatureCollection",
  "features": [
  """)
for i in range(100):
    idx = color_area.index(idx_color2[str(i)][4])
    colorful = map_color[str(idx)]
    #print(colorful)
    if colorful != "white":
        latlong = ""
        #for j in range(4):
        #    latlong += str(list(idx_color2[str(i)][j])) +", "
        latlong += str(list(idx_color2[str(i)][0])) +", "
        latlong += str(list(idx_color2[str(i)][1])) +", "
        latlong += str(list(idx_color2[str(i)][3])) +", "
        latlong += str(list(idx_color2[str(i)][2])) +", "
        latlong += str(list(idx_color2[str(i)][0]))
        #witf.write("{location: new google.maps.LatLng(%f, %f)},\n" % (test_kyo1['ex_lat'][i],test_kyo1['ex_long'][i]))
        f_loc.write("""{
          "type": "Feature",
          "properties": {
            "letter": "o",
            "color": "%s",
            "rank": "15",
            "ascii": "111"
          },
          "geometry": {
            "type": "Polygon",
            "coordinates": [
              [
                %s 
              ]
            ]
          }
        },
        """ % (colorful, latlong))
    # remember to delete last ","
f_loc.write("""]
}""")
f_loc.close()


-1 -1 -1  3 -1  5  5  7  7  7 
-1 11 -1  3  3  3 16 16 16 16 
-1 11 11  3  3  3  3  3 16 16 
-1 -1 -1  3  3  3  3 -1 -1 16 
-1 -1 42 42 42 42 42 42 42 -1 
-1 51 42 42 42 42 42 42 42 42 
-1 -1 -1 63 63 63 -1 -1 42 42 
-1 -1 -1 63 63 63 63 77 -1 42 
-1 -1 -1 63 63 63 63 63 63 63 
-1 -1 92 92 63 63 63 -1 63 63 

 #areas : 11
